In [ ]:
# Setup dan Import
!pip install -q google-cloud-aiplatform
!pip install -q google-cloud-monitoring
!pip install -q google-cloud-bigquery
!pip install -q pandas numpy plotly

import os
from google.cloud import monitoring_v3
from google.cloud import bigquery
from google.cloud import aiplatform
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta

In [ ]:
# Setup project
PROJECT_ID = "your-project-id"
REGION = "your-region"
ENDPOINT_NAME = "air-quality-model"

In [ ]:
# Setup monitoring client
client = monitoring_v3.MetricServiceClient()
project_name = f"projects/{PROJECT_ID}"

In [ ]:
# Function untuk mengambil metrics dari Cloud Monitoring
def get_metric_data(metric_type, hours=24):
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(hours=hours)
    
    interval = monitoring_v3.TimeInterval({
        'start_time': start_time,
        'end_time': end_time,
    })
    
    results = client.list_time_series(
        request={
            "name": project_name,
            "filter": f'metric.type = "{metric_type}"',
            "interval": interval,
            "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL
        }
    )
    
    return results

In [ ]:
# Function untuk evaluasi model performance
def evaluate_model_performance():
    # Query BigQuery untuk membandingkan prediksi dengan actual values
    bq_client = bigquery.Client()
    
    query = """
    SELECT
        timestamp,
        device_id,
        raw_pm25,
        calibrated_pm25,
        raw_pm10,
        calibrated_pm10,
        raw_o3,
        calibrated_o3,
        raw_co,
        calibrated_co,
        raw_no2,
        calibrated_no2
    FROM
        `air_quality.measurements`
    WHERE
        timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 24 HOUR)
    ORDER BY
        timestamp DESC
    """
    
    df = bq_client.query(query).to_dataframe()
    return df

In [ ]:
# Function untuk visualisasi metrics
def plot_metrics(df, parameter):
    fig = go.Figure()
    
    # Plot raw vs calibrated values
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df[f'raw_{parameter}'],
        name='Raw Data',
        mode='lines+markers'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df[f'calibrated_{parameter}'],
        name='Calibrated Data',
        mode='lines+markers'
    ))
    
    fig.update_layout(
        title=f'{parameter.upper()} - Raw vs Calibrated Values',
        xaxis_title='Timestamp',
        yaxis_title='Value',
        hovermode='x unified'
    )
    
    return fig

In [ ]:
# Function untuk deteksi anomali
def detect_anomalies(df, parameter, threshold=3):
    """Detect anomalies using Z-score method"""
    raw_col = f'raw_{parameter}'
    cal_col = f'calibrated_{parameter}'
    
    # Calculate z-scores
    z_scores = np.abs((df[cal_col] - df[cal_col].mean()) / df[cal_col].std())
    
    # Mark anomalies
    anomalies = df[z_scores > threshold].copy()
    return anomalies

In [ ]:
# Function untuk generate monitoring report
def generate_monitoring_report():
    # Get performance data
    df = evaluate_model_performance()
    
    print("Model Performance Report")
    print("-" * 50)
    
    # Analyze each parameter
    parameters = ['pm25', 'pm10', 'o3', 'co', 'no2']
    
    for param in parameters:
        print(f"\nAnalysis for {param.upper()}:")
        
        # Calculate basic statistics
        raw_col = f'raw_{param}'
        cal_col = f'calibrated_{param}'
        
        raw_mean = df[raw_col].mean()
        cal_mean = df[cal_col].mean()
        
        print(f"Average Raw Value: {raw_mean:.2f}")
        print(f"Average Calibrated Value: {cal_mean:.2f}")
        
        # Detect anomalies
        anomalies = detect_anomalies(df, param)
        print(f"Number of anomalies detected: {len(anomalies)}")
        
        # Create visualization
        fig = plot_metrics(df, param)
        fig.show()

In [ ]:
# Setup alert policies
def setup_alert_policies():
    from google.cloud.monitoring_v3 import AlertPolicy
    from google.cloud.monitoring_v3 import AlertPolicyService
    
    alert_client = AlertPolicyService()
    
    # Create alert policy for model performance
    alert_policy = AlertPolicy({
        'display_name': 'Air Quality Model Performance Alert',
        'conditions': [{
            'display_name': 'High Error Rate',
            'condition_threshold': {
                'filter': 'metric.type="custom.googleapis.com/air_quality/error_rate"',
                'comparison': AlertPolicy.ComparisonType.COMPARISON_GT,
                'threshold_value': 0.2,
                'duration': {'seconds': 300},  # 5 minutes
            }
        }],
        'notification_channels': [
            f"projects/{PROJECT_ID}/notificationChannels/{NOTIFICATION_CHANNEL_ID}"
        ],
        'enabled': True
    })
    
    created_policy = alert_client.create_alert_policy(
        request={
            "name": project_name,
            "alert_policy": alert_policy
        }
    )
    
    return created_policy

In [ ]:
# Main monitoring dashboard
def main():
    print("Starting Air Quality Model Monitoring")
    print("=" * 50)
    
    # Generate performance report
    generate_monitoring_report()
    
    # Setup alerts if they don't exist
    try:
        alert_policy = setup_alert_policies()
        print("\nAlert policies setup successfully")
    except Exception as e:
        print(f"\nError setting up alert policies: {str(e)}")
    
    print("\nMonitoring dashboard initialized successfully")

In [ ]:
# Run monitoring dashboard
if __name__ == "__main__":
    main()